In [ ]:
from datetime import date

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [ ]:
# ============================================================
# ИСХОДНЫЕ ДАННЫЕ (на основе фактически выполненных фаз)
# ============================================================

# Фаза 2: 18 ч за 4 недели → 4.5 ч/нед
# Фаза 3: 12 ч за 3 недели → 4.0 ч/нед
# Используем это как дискретное распределение
man_hours_per_week_history = [4.0, 4.5]

# Производительность по задачам (оценено вручную)
# Фаза 2 и 3: условно 1–2 задачи в неделю
tasks_done_per_week_history = [1, 2]

# Количество задач Фазы 5
tasks_count = 5

# ============================================================
# ТРЁХТОЧЕЧНЫЕ ОЦЕНКИ ЗАДАЧ ФАЗЫ 5 (PERT)
# ============================================================

# (O, BG, P)
# tasks_triangulars = [
#     (1.5, 2.5, 4.0),   # Декомпозиция проекта
#     (1.0, 1.5, 2.5),   # Формирование команды
#     (2.0, 3.0, 4.5),   # Ресурсные требования
#     (2.0, 3.0, 5.0),   # Связи предшествования
#     (2.5, 4.0, 6.0),   # Эвристическое планирование
# ]
# (O, BG, P)

tasks_triangulars = [
    (1.5, 2.5, 3.0),   # Декомпозиция проекта
    (1.0, 1.5, 2.5),   # Формирование команды
    (2.0, 3.0, 4.5),   # Ресурсные требования
    (2.0, 3.0, 4.5),   # Связи предшествования
    (2, 3.5, 4.0),   # Эвристическое планирование
]

# ============================================================
# ВСПОМОГАТЕЛЬНЫЕ ФУНКЦИИ
# ============================================================

def analyze_results(results, confidence=0.8):
    """Возвращает среднее и доверительный интервал"""
    low = np.percentile(results, (1 - confidence) / 2 * 100)
    high = np.percentile(results, (1 + confidence) / 2 * 100)
    return np.mean(results), low, high

# ============================================================
# МОДЕЛЬ 1: Монте-Карло по количеству задач в неделю
# ============================================================

iterations = 10000
weeks_result_mc1 = []

for _ in range(iterations):
    completed_tasks = 0
    weeks = 0

    while completed_tasks < tasks_count:
        weeks += 1
        completed_tasks += np.random.choice(tasks_done_per_week_history)

    weeks_result_mc1.append(weeks)

weeks_result_mc1 = np.array(weeks_result_mc1)
mean1, low1, high1 = analyze_results(weeks_result_mc1)

print("Модель 1 (по количеству задач):")
print(f"Средний срок: {mean1:.2f} недель")
print(f"80% интервал: [{low1}; {high1}] недель")

# ============================================================
# МОДЕЛЬ 2: Монте-Карло по трудозатратам (человеко-часы)
# ============================================================

weeks_result_mc2 = []

for _ in range(iterations):
    # Сэмплируем суммарную трудоёмкость Фазы 5
    total_work = sum(np.random.triangular(o, bg, p) for o, bg, p in tasks_triangulars)

    completed_work = 0
    weeks = 0

    while completed_work < total_work:
        weeks += 1
        completed_work += np.random.choice(man_hours_per_week_history)

    weeks_result_mc2.append(weeks)

weeks_result_mc2 = np.array(weeks_result_mc2)
mean2, low2, high2 = analyze_results(weeks_result_mc2)

print("\nМодель 2 (по трудозатратам):")
print(f"Средний срок: {mean2:.2f} недель")
print(f"80% интервал: [{low2}; {high2}] недель")


Модель 1 (по количеству задач):
Средний срок: 3.56 недель
80% интервал: [3.0; 4.0] недель

Модель 2 (по трудозатратам):
Средний срок: 3.77 недель
80% интервал: [3.0; 4.0] недель
